In [2]:
import numpy as np
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
from tqdm import tqdm
import random

In [3]:
maxlen = 100
kernel_size = 3
batch_size = 32
embedded_size = 128

In [4]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [5]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [22, 2723, 18, 1122, 250, 170, 58, 4, 1398, 7006] ['never', 'laughoutloud', 'funny', 'frequently', 'amusing', 'fascinating', 'fun', 'film', 'philosophical', 'musings']


In [6]:
idx_trainset = []
for text in trainset.data:
    idx = []
    for t in text.split():
        try:
            idx.append(dictionary[t])
        except:
            pass
    idx_trainset.append(idx)

In [7]:
X = tf.keras.preprocessing.sequence.pad_sequences(idx_trainset, maxlen)
X.shape

(10662, 100)

In [8]:
train_X, test_X, train_Y, test_Y = train_test_split(X, 
                                                    trainset.target,
                                                    test_size = 0.2)

In [9]:
def gated_linear_unit(x, d_rate):
    c = tf.layers.conv1d(inputs = x,
                         filters = 2*embedded_size,
                         kernel_size = kernel_size,
                         dilation_rate = d_rate,
                         padding = 'same')
    c1, c2 = tf.split(c, 2, -1)
    x += c1 * tf.sigmoid(c2)
    return x

class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        for d_rate in [1, 2, 4]:
            encoder_embedded = gated_linear_unit(encoder_embedded, d_rate)
    
        encoder_embedded = tf.reduce_max(encoder_embedded, 1)
        encoder_embedded = tf.layers.flatten(encoder_embedded)
        forward = tf.layers.dense(encoder_embedded, embedded_size, tf.nn.relu)
        self.logits = tf.layers.dense(forward, dimension_output)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,2,1e-3)
sess.run(tf.global_variables_initializer())

In [11]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : train_X[i:i+batch_size], model.Y : train_Y[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : test_X[i:i+batch_size], model.Y : test_Y[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.639678
time taken: 3.434479236602783
epoch: 0, training loss: 0.675803, training acc: 0.586231, valid loss: 0.631852, valid acc: 0.639678

epoch: 1, pass acc: 0.639678, current acc: 0.662405
time taken: 1.630638599395752
epoch: 1, training loss: 0.298692, training acc: 0.884868, valid loss: 0.727101, valid acc: 0.662405

epoch: 2, pass acc: 0.662405, current acc: 0.676136
time taken: 1.631056308746338
epoch: 2, training loss: 0.057012, training acc: 0.983670, valid loss: 1.020313, valid acc: 0.676136

time taken: 1.588785171508789
epoch: 3, training loss: 0.025513, training acc: 0.991424, valid loss: 1.363197, valid acc: 0.663826

time taken: 1.6412477493286133
epoch: 4, training loss: 0.030138, training acc: 0.990484, valid loss: 1.669384, valid acc: 0.645360

time taken: 1.5997257232666016
epoch: 5, training loss: 0.108270, training acc: 0.966870, valid loss: 1.447906, valid acc: 0.675663

epoch: 6, pass acc: 0.676136, current acc: 0.68797

In [12]:
logits = sess.run(model.logits, feed_dict={model.X:test_X})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.68      0.66      0.67      1083
    positive       0.66      0.68      0.67      1050

    accuracy                           0.67      2133
   macro avg       0.67      0.67      0.67      2133
weighted avg       0.67      0.67      0.67      2133

